<a href="https://colab.research.google.com/github/jamg-upv/Impacto-citas-articulo/blob/main/impactoencitas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

How to cite this work:

------------

Este codigo permite integrar una tabla con los datos de un conjunto de articulos y sus numeros de citas, normalizarlas teniendo en cuenta las tablas de citas mundiales (scopus) o baseline de (WOS), y añadir las métricas de impacto de la revista en el año de publicación del articulo.
Son necesarias 3 tablas para cada proveedor (scopus y WOS)


Fuente de datos SCOPUS https://elsevier.com/?a=734751


Tareas 
1. Lista de 25 DOIs Banco de Pruebas1  (meritos (5 autores "al azar") )
2. Descargar como cvs bancopruebas1sco tabla Scopus  de 25 meritos (5 autores "al azar") a partir de los DOI
1. Descargar como cvs bancopruebas1wos tabla WOS de los 25 meritos a partir de los DOI
1. preparar un prototipo de la tabla que penamos crear para consultar a compañeros del equipo
2.   



Pasos previos. Librerias y configuración

In [ ]:
# Importamos ls librería de Python que necesitaremos en este notebook

import pandas as pd  # Pandas permite manipular tablas 
import matplotlib.pyplot as plt  # Para hacer gráficas
import seaborn as sns  # Para hacer gráficas pero necesita matplotlib y deja unos graficos chulisismos 
sns.set_style('whitegrid')  # Pone una malla de fondo bastante mona para Seaborn
#import xlwings as xw #permite leer docuemtnos xlb me da error y no lo puedo instalar

# Redondeamos las salidas a dos decimales
%precision %.2f

# from google.colab import drive
# drive.mount('/content/drive')



Carga de *datos* Indicadores de revistas SCOPUS

In [11]:
#métricas de revistas Scopus
# si lo descargamos directo de la web Ç(ventaja es que con una descarga siempre está accesible y no hay que volver a descargarlo durante la sesión)
# el archivo de SCopus de 10 años ocupa demasiados GB (60gb) y no se puede alojar en drive (no debería esar en un acceso libre en github porque es propiedad de Scopus)
   #!wget https://elsevier.com/?a=734751 
#es XLB, hay que convertirlo a mano XLS porque xlb requiere de librerias que no tengo instaladas en colab
# y hacerlo accesible aquí
#la primer pesaña son definiciones de indicadores, la segunra el el año más reciente y luego 9 mas con los anteriores. La ultima pestaña es un clasificación de categorias
## limpio a mano las pestañas no usadas (priemera y ultima) y dejo solo los 7 años más recientes (para que sean solo 44mb y me quepa en google drive)
!wget https://docs.google.com/spreadsheets/d/1gY0gVLoYxCN522Z_6WpzggG3IVodJlMBxhE8J_b__mg/edit?usp=sharing

--2022-02-07 19:04:29--  https://docs.google.com/spreadsheets/d/1gY0gVLoYxCN522Z_6WpzggG3IVodJlMBxhE8J_b__mg/edit?usp=sharing
Resolving docs.google.com (docs.google.com)... 142.250.103.100, 142.250.103.102, 142.250.103.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.103.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘edit?usp=sharing’

edit?usp=sharing        [ <=>                ] 331.04K  --.-KB/s    in 0.1s    

2022-02-07 19:04:32 (2.53 MB/s) - ‘edit?usp=sharing’ saved [338984]



In [12]:
# Creamos una tabla con Pandas para cada uno de los ficheros
df = pd.read_excel('DataScopus.xlsb', engine='pyxlsb')





ImportError: ignored